In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import re

tables = {
    "Opening": np.zeros((6, 8, 8), dtype=int),
    "Endgame": np.zeros((6, 8, 8), dtype=int)
}
fig_names = ["Pion", "Skoczek", "Goniec", "Wieża", "Hetman", "Król"]
current_phase = "Opening"
current_piece = 0
current_value = 0
click_enabled = True

def parse_eval_tables_from_file(filename):
    with open(filename, 'r') as f:
        content = f.read()

    pattern = re.compile(
        r'const int Eval::Pieces(Opening|Endgame)PositionTable\[\d\]\[64\] = \{(.*?)\};',
        re.DOTALL
    )
    matches = pattern.findall(content)

    for phase, table_text in matches:
        arrays = re.findall(r'\{([^{}]+)\}', table_text)
        for i, arr in enumerate(arrays):
            nums = list(map(int, arr.strip().replace('\n', '').split(',')))
            tables[phase][i] = np.array(nums).reshape(8, 8)

parse_eval_tables_from_file("EvalLookup.cpp")

value_input = widgets.IntText(description="Wartość:", value=0)
phase_dropdown = widgets.Dropdown(options=["Opening", "Endgame"], description="Faza gry:")
piece_dropdown = widgets.Dropdown(options=fig_names, description="Figura:")
save_button = widgets.Button(description="Zapisz do pliku", button_style='success')

def on_value_change(change): global current_value; current_value = change['new']
def on_phase_change(change): global current_phase; current_phase = change['new']; update_plot()
def on_piece_change(change): global current_piece; current_piece = fig_names.index(change['new']); update_plot()

value_input.observe(on_value_change, names='value')
phase_dropdown.observe(on_phase_change, names='value')
piece_dropdown.observe(on_piece_change, names='value')

fig, ax = plt.subplots(figsize=(6,6))

def update_plot():
    global fig, ax, heatmap

    fig.clf()
    ax = fig.add_subplot(1, 1, 1)

    matrix = tables[current_phase][current_piece]
    heatmap = ax.imshow(matrix[::-1], cmap='coolwarm', vmin=-50, vmax=50)

    ax.set_xticks(range(8))
    ax.set_yticks(range(8))
    ax.set_xticklabels(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
    ax.set_yticklabels(list(range(1, 9))[::-1])
    ax.set_title(f"{fig_names[current_piece]} ({current_phase})")

    fig.colorbar(heatmap, ax=ax)
    fig.canvas.draw_idle()
    fig.canvas.mpl_connect('button_press_event', on_click)  # Rebind click

def on_click(event):
    if not event.inaxes or not click_enabled:
        return
    x, y = int(event.xdata), 7 - int(event.ydata)
    if 0 <= x < 8 and 0 <= y < 8:
        tables[current_phase][current_piece][y][x] = current_value
        update_plot()


fig.canvas.mpl_connect('button_press_event', on_click)

def save_to_file(_):
    with open("updated_eval_tables.h", "w") as f:
        for phase in ["Opening", "Endgame"]:
            f.write(f"const int Eval::Pieces{phase}PositionTable[6][64] = {{\n")
            for i, piece_table in enumerate(tables[phase]):
                f.write("    {")
                flat = piece_table.flatten()
                for j in range(64):
                    val = flat[j]
                    f.write(f"{val:>4}")
                    if j < 63:
                        f.write(",")
                    if (j + 1) % 8 == 0:
                        f.write("\n     ")
                f.write("},\n\n")
            f.write("};\n\n")
    print("Zapisano do: updated_eval_tables.h")

save_button.on_click(save_to_file)

display(widgets.HBox([value_input, phase_dropdown, piece_dropdown]))
display(save_button)
update_plot()
plt.show()


: 